# Tutorial

In [2]:
import os
import scanpy as sc
import pickle
import sys
from collections import Counter
sys.path.append("../..")

In [3]:
from idtrack import *
from idtrack._track_tests import *

In [4]:
logger_config()
local_dir = "/lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp"  # or any other local directory
dm = DatabaseManager("homo_sapiens", 107, "gene", local_dir, 79) 

In [5]:
tt: TrackTests = TrackTests(dm)

2022-08-16 21:48:15 INFO:graph: The graph is being read.


In [6]:
base_path = "/lustre/groups/ml01/workspace/hlca_lisa.sikkema_malte.luecken/HLCA_reproducibility/data"
dset0_dir = os.path.join(base_path, "HLCA_extended/extension_datasets/ready/full")
dset1_dir = os.path.join(base_path, "HLCA_extended/extension_datasets/raw")

adata_dict = {
    "Kaminski_2020": [f"{dset0_dir}/adams.h5ad"],
    "Meyer_2021": [f"{dset0_dir}/meyer_2021.h5ad"],
    "MeyerNikolic_unpubl": [f"{dset0_dir}/meyer_nikolic_unpubl.h5ad"],
    "Barbry_unpubl": [f"{dset0_dir}/barbry.h5ad"],
    "Regev_2021": [
        f"{dset0_dir}/delorey_cryo.h5ad", f"{dset0_dir}/delorey_fresh.h5ad",
        f"{dset0_dir}/delorey_nuclei.h5ad"
    ],
    "Thienpont_2018": [f"{dset1_dir}/Lambrechts/lambrechts.h5ad"],
    "Budinger_2020": [f"{dset0_dir}/bharat.h5ad"],
    "Banovich_Kropski_2020": [f"{dset0_dir}/haberman.h5ad"],
    "Sheppard_2020": [f"{dset0_dir}/tsukui.h5ad"],
    "Wunderink_2021": [
        f"{dset0_dir}/grant_cryo.h5ad", f"{dset0_dir}/grant_fresh.h5ad"
    ],
    "Lambrechts_2021": [
        f"{dset0_dir}/wouters.h5ad" #, f"{dset0_dir}/wouters_labs.h5ad"
    ],
    "Zhang_2021": [f"{dset1_dir}/Liao/covid_for_publish.h5ad"],
    "Duong_lungMAP_unpubl": [f"{dset0_dir}/duong.h5ad"],
    "Janssen_2020": [f"{dset0_dir}/mould.h5ad"],
    "Sun_2020": [
        f"{dset0_dir}/wang_sub_batch1.h5ad", f"{dset0_dir}/wang_sub_batch2.h5ad",
        f"{dset0_dir}/wang_sub_batch3.h5ad", f"{dset0_dir}/wang_sub_batch4.h5ad"],
    "Gomperts_2021": [
        f"{dset0_dir}/carraro_ucla.h5ad", f"{dset0_dir}/carraro_cff.h5ad",
        f"{dset0_dir}/carraro_csmc.h5ad"],
    "Eils_2020": [f"{dset0_dir}/lukassen.h5ad"],
    "Schiller_2020": [f"{dset0_dir}/mayr.h5ad"],
    "Misharin_Budinger_2018": [f"{dset0_dir}/reyfman_disease.h5ad"],
    "Shalek_2018": [f"{dset0_dir}/ordovasmontanes.h5ad"],
    "Schiller_2021": [f"{dset0_dir}/schiller_discovair.h5ad"],
    "Peer_Massague_2020": [f"{dset0_dir}/laughney.h5ad"],
    "Lafyatis_2019": [f"{dset0_dir}/valenzi.h5ad"],
    "Tata_unpubl": [f"{dset0_dir}/tata_unpubl.h5ad"],
    "Xu_2020": [f"{dset0_dir}/guo.h5ad"],
    "Sims_2019": [f"{dset0_dir}/szabo.h5ad"],
    "Schultze_unpubl": [f"{dset0_dir}/schultze_unpubl.h5ad"]
}

In [6]:
base_path = "/lustre/groups/ml01/workspace/hlca_lisa.sikkema_malte.luecken/HLCA_reproducibility/data"
dset0_dir = os.path.join(base_path, "HLCA_extended/extension_datasets/ready/full")
dset1_dir = os.path.join(base_path, "HLCA_extended/extension_datasets/raw")

adata_dict = {
    "Kaminski_2020": [f"{dset0_dir}/adams.h5ad"],
    "Eils_2020": [f"{dset0_dir}/lukassen.h5ad"],
    "Sims_2019": [f"{dset0_dir}/szabo.h5ad"],
    "Schultze_unpubl": [f"{dset0_dir}/schultze_unpubl.h5ad"],
}

#with open(os.path.join(local_dir, "results_databases.pk"), 'rb') as handle:
#    result = pickle.load(handle)
#    print("Loaded")

In [7]:
result = dict()
for ddadaata in adata_dict:
    
    adata = sc.read(adata_dict[ddadaata][0])
    gene_list = list(adata.var.index)
    gene_list_filtered, gene_list_converted, gene_list_lost = tt.unfound_correct(list(adata.var.index))
    
    gene_list = set(gene_list)
    print(ddadaata, len(gene_list_filtered), len(gene_list_filtered)-len(set(gene_list_filtered)))
    gene_list_filtered = set(gene_list_filtered)
   
    for i in gene_list_filtered:
        if i not in tt.graph.nodes:
            raise ValueError
    print(tt.identify_source(gene_list_filtered)[0])
    
    a = dict()
    for ind, i in enumerate(gene_list_filtered):
        if ind % 100 == 0 or ind > len(gene_list_filtered)-5:
            progress_bar(ind, len(gene_list_filtered)-1)
        conv = tt.convert(i, None, None, 'ensembl_gene', prioritize_to_one_filter=True, return_path=True)
        a[i] = conv
    result[ddadaata] = a
    print(list(Counter([len(a[i]) if a[i] is not None else 0 for i in a]).most_common())[:10])
    print()

2022-08-16 21:49:00 WARNING:track: Number of converted IDs with small modifications: 355


Kaminski_2020 45947 0
(('HGNC Symbol', 94), 30503)
Progress: |████████████████████| 100.0% 
[(1, 45536), (0, 411)]



2022-08-16 21:51:55 WARNING:track: Number of converted IDs with small modifications: 671
2022-08-16 21:51:55 WARNING:track: Number of IDs not found in the graph: 7


Eils_2020 32731 99
(('Vega gene', 80), 30093)
Progress: |████████████████████| 100.0% 
[(1, 31742), (0, 888), (3, 1), (2, 1)]



2022-08-16 21:53:00 WARNING:track: Number of converted IDs with small modifications: 2360
2022-08-16 21:53:00 WARNING:track: Number of IDs not found in the graph: 10


Sims_2019 60715 1894
(('Vega gene', 84), 42737)
Progress: |████████████████████| 100.0% 
[(1, 57091), (0, 1723), (2, 6), (3, 1)]



2022-08-16 21:57:37 WARNING:track: Number of converted IDs with small modifications: 309


Schultze_unpubl 24532 1
(('HGNC Symbol', 91), 19500)
Progress: |████████████████████| 100.0% 
[(1, 24439), (0, 91), (2, 1)]



In [8]:
### Check how many total targets are 

In [8]:
only_matches = {ddad: {k: list(i.keys()) if i is not None else [] for k, i in result[ddad].items()} for ddad in adata_dict}
all_targets = {k for _, i in only_matches.items() for _, j in i.items() for k in j}
all_targets = sorted(all_targets)
len(all_targets)

56870

In [9]:
for ddd in only_matches:
    db_1to1 = [l for c, t in only_matches[ddd].items() for l in t if len(t) == 1]
    print(ddd, '\t','\t', len(db_1to1), len(set(db_1to1)), len(db_1to1)-len(set(db_1to1)))

Kaminski_2020 	 	 45536 45356 180
Eils_2020 	 	 31742 31225 517
Sims_2019 	 	 57091 55269 1822
Schultze_unpubl 	 	 24439 24231 208


In [10]:
klk = [i for i in db_1to1 if db_1to1.count(i) > 1]
klk = np.unique(klk)
print(len(klk))
print(klk[0:5])

194
['ENSG00000007129.18' 'ENSG00000011021.23' 'ENSG00000033011.14'
 'ENSG00000067601.10' 'ENSG00000071127.18']


In [11]:
for jjj in range(20,40):
    lll = [(c,t) for c, t in only_matches[ddd].items() if klk[jjj] in t]
    for llll in [i for i, j in lll]:
        print(tt.convert(llll, None, None, 'ensembl_gene', return_path=True, prioritize_to_one_filter=True))
    print()

{'ENSG00000150337.14': [-1, -1, nan, 2, 0, 4, (('FCGR1A', 'ENSG00000150337.14', 0, ('ensembl_gene', 103)),)]}
{'ENSG00000150337.14': [-1, -3, nan, 2, 0, 4, (('FCGR1B', 'ENSG00000266332.1', 0, ('assembly_37_ensembl_gene', 103)), ('ENSG00000266332.1', '2209', 0, ('external', 103)), ('2209', 'ENSG00000150337.14', 0, ('ensembl_gene', 103)))]}

{'ENSG00000151338.21': [-1, -1, nan, 2, 0, 30, (('MIPOL1', 'ENSG00000151338.21', 0, ('ensembl_gene', 107)),)]}
{'ENSG00000151338.21': [-1, -1, 0.99, 2, 0, 30, (('AL121790.1', 'ENSG00000258414.1', 0, ('ensembl_gene', 105)), ('ENSG00000258414.1', 'ENSG00000151338.20', 0), ('ENSG00000151338.20', 'ENSG00000151338.21', 0))]}

{'ENSG00000156170.14': [-1, -1, nan, 2, 0, 33, (('NDUFAF6', 'ENSG00000156170.14', 0, ('ensembl_gene', 107)),)]}
{'ENSG00000156170.14': [-1, -1, 0.99, 2, 0, 33, (('AC087752.3', 'ENSG00000253878.5', 0, ('ensembl_gene', 106)), ('ENSG00000253878.5', 'ENSG00000156170.14', 0))]}

{'ENSG00000157654.19': [-1, -1, nan, 5, 0, 18, (('PALM2', 'E

In [11]:
ddad = "Schultze_unpubl"
only_matches = {ddad: {k: list(i.keys()) if i is not None else [] for k, i in result[ddad].items()}}
for ddd in only_matches:
    db_1to1 = [l for c, t in only_matches[ddd].items() for l in t if len(t) == 1]
    print(ddd, '\t','\t', len(db_1to1), len(set(db_1to1)), len(db_1to1)-len(set(db_1to1)))
klk = [i for i in db_1to1 if db_1to1.count(i) > 1]
np.array(klk[:10])

Schultze_unpubl 	 	 24348 24128 220


array(['ENSG00000229951.6', 'ENSG00000122674.12', 'ENSG00000246308.4',
       'ENSG00000276463.4', 'ENSG00000238258.1', 'ENSG00000008128.23',
       'ENSG00000215068.10', 'ENSG00000265190.7', 'ENSG00000267481.1',
       'ENSG00000197061.5'], dtype='<U18')

In [89]:
ddadaata = "Schultze_unpubl"
adata = sc.read(adata_dict[ddadaata][0])
gene_list = list(adata.var.index)
gene_list_filtered, gene_list_converted, gene_list_lost = tt.unfound_correct(list(adata.var.index))

gene_list = set(gene_list)
print(ddadaata, len(gene_list_filtered), len(gene_list_filtered)-len(set(gene_list_filtered)))
gene_list_filtered = set(gene_list_filtered)

for i in gene_list_filtered:
    if i not in tt.graph.nodes:
        raise ValueError
print(tt.identify_source(gene_list_filtered)[0])

a = dict()
for ind, i in enumerate(gene_list_filtered):
    if ind % 100 == 0 or ind > len(gene_list_filtered)-5:
        progress_bar(ind, len(gene_list_filtered)-1)
    conv = tt.convert(i, None, None, 'ensembl_gene', prioritize_to_one_filter=False, return_path=False)
    a[i] = conv
result[ddadaata] = a
print(list(Counter([len(a[i]) if a[i] is not None else 0 for i in a]).most_common())[:10])
print()
with open(os.path.join(local_dir, "results_databases_no_prioritize.pk"), 'wb') as handle:
    pickle.dump(result, handle)
    print("Saved")

2022-08-16 13:05:46 WARNING:track: Number of converted IDs with small modifications: 444


Schultze_unpubl 24532 1
(('HGNC Symbol', 107), 19514)
Progress: |████████████████████| 100.0% 
[(1, 18709), (2, 2800), (3, 1059), (4, 587), (5, 336), (6, 187), (7, 178), (8, 146), (9, 94), (10, 91)]



In [29]:
with open(os.path.join(local_dir, "results_databases_no_prioritize.pk"), 'rb') as handle:
    result = pickle.load(handle)
    print("Loaded")

Loaded


In [30]:
# ignore all 0's
# consume all 1-to-1s

# cakisik aileleri bul
    # 1-to-n's that at least one n id the same

# for each cakisik aile
    # nested_for_loop: asagidaki gibi, minimum olani sec.

In [31]:
import networkx as nx
g = nx.Graph()
for i in result["Schultze_unpubl"].keys():
    if result["Schultze_unpubl"][i]:
        for j in result["Schultze_unpubl"][i].keys():
            g.add_edge(f"Source::{i}", f"Target::{j}")

In [32]:
consumed = {klk.split("Target::")[1] for kl in [i for i in list(nx.connected_components(g)) if len(i) == 2] for klk in kl if klk.startswith("Target::")}

In [33]:
temp_ = [i for i in list(nx.connected_components(g)) if len(i) > 2]
tp = [t for t in temp_ if sum([i.startswith("Source::") for i in t]) > 1] # many-to-1 or many-to-many
#tp=[[j.split("::")[1] for j in i] for i in tp]

In [66]:
len(list(nx.connected_components(g))), len(temp_), len(tp)

(20377, 2421, 1555)

In [34]:
# [len(i) for i in tp]
[klk for klk in tp[0] if klk.startswith("Target::") and klk.split("Target::")[1] in consumed]

[]

In [35]:
playground=tp[0]

asd = list()
for i in playground:
    j, k = i.split("::")
    if j == "Source":
        asd.append(len(result["Schultze_unpubl"][k]))

In [53]:
asdad = [g.degree(i) for i in playground if i.startswith("Target::")]
len([i for i in asdad if i!= 1]), len(asdad), len([g.degree(i) for i in playground if i.startswith("Source::")])

(187, 251, 164)

In [87]:
amk = {i: list(g.neighbors(i)) for i in playground if i.startswith("Target::")}

amkr = dict()
for i in amk:
    for j in amk[i]:
        if j not in amkr:
            amkr[j] = [i]
        else:
            amkr[j].append(i)

In [94]:
{i: s for i in amkr if len(amkr[i]) == 1 for s in list(g.neighbors(i))}

{'Source::MOCOS': 'Target::ENSG00000075643.6',
 'Source::ADRA1B': 'Target::ENSG00000170214.5',
 'Source::AC016876.2': 'Target::ENSG00000264772.6',
 'Source::PSMD7P1': 'Target::ENSG00000266707.1',
 'Source::PMAIP1': 'Target::ENSG00000141682.12',
 'Source::PAG1': 'Target::ENSG00000076641.4',
 'Source::ANOS1': 'Target::ENSG00000011201.12',
 'Source::APOBEC3C': 'Target::ENSG00000244509.4',
 'Source::TIMP3': 'Target::ENSG00000100234.12',
 'Source::EP300': 'Target::ENSG00000100393.14',
 'Source::APOBEC2': 'Target::ENSG00000124701.6',
 'Source::SELENOP': 'Target::ENSG00000250722.6',
 'Source::CRK': 'Target::ENSG00000167193.8',
 'Source::GJA1': 'Target::ENSG00000152661.9',
 'Source::GFER': 'Target::ENSG00000127554.13',
 'Source::PRDX1P1': 'Target::ENSG00000214121.4'}

In [97]:
x = dm.get_table("xref", usecols=["xref_id", "external_db_id", "dbprimary_acc", "display_label"],  save_after_calculation=False)

In [108]:
es = dm.get_table("external_synonym", usecols=["xref_id", "synonym"], save_after_calculation=False)
esm = es.merge(x, how="inner", on="xref_id", validate="many_to_one")
esm.drop(columns=["display_label"], inplace=True)
esm.rename(columns={"synonym": "display_label"}, inplace=True)

In [153]:
dm.change_server(38).clean_up([i for i in dm.change_server(38).tables_in_disk() if 'processed_external' in i])

/home/icb/kemal.inecik/work/codes/idtrack/docs/_notebooks/../../idtrack/_database_manager.py:1080: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['old_stable_id', 'new_stable_id', 'type'], dtype='object', name=0)]

  df.to_hdf(file_path, key=hierarchy, mode="a", **self._comp_hdf5)


In [154]:
dm.change_server(37).clean_up([i for i in dm.change_server(37).tables_in_disk() if 'processed_external' in i])

In [55]:
the_id_1, the_id_2, the_id_3 = "ROR1", "ROR2", "RORA"
print(tt.convert(the_id_1, None, None, 'ensembl_gene', return_path=False, prioritize_to_one_filter=True))
print(tt.convert(the_id_2, None, None, 'ensembl_gene', return_path=False, prioritize_to_one_filter=True))
print(tt.convert(the_id_3, None, None, 'ensembl_gene', return_path=False, prioritize_to_one_filter=True))

{'ENSG00000069667.16': [-1, -1, 0.9961195, 7, 0, 14]}
{'ENSG00000069667.16': [-1, -1, 0.9961195, 7, 0, 14]}
{'ENSG00000069667.16': [-1, -1, 0.9961195, 7, 0, 14]}


In [56]:
a1=tt.convert(the_id_1, None, None, 'ensembl_gene', return_path=False, prioritize_to_one_filter=False)
a1
## bu neden 1 varken 2'yi de aldi amk

{'ENSG00000069667.16': [1, 1, 0.9961195, 4],
 'ENSG00000169071.15': [1, 3, 0.9767402500000001, 4],
 'ENSG00000185483.13': [1, 1, 0.9950012000000001, 5]}

In [57]:
a2=tt.convert(the_id_2, None, None, 'ensembl_gene', return_path=False, prioritize_to_one_filter=False)
a2
## bu neden 1 varken 2'yi de aldi amk

{'ENSG00000169071.15': [1, 1, 0.9767402500000001, 4],
 'ENSG00000185483.13': [1, 3, 0.9950012000000001, 5],
 'ENSG00000069667.16': [1, 1, 0.9961195, 4]}

In [58]:
a3=tt.convert(the_id_3, None, None, 'ensembl_gene', return_path=False, prioritize_to_one_filter=False)
a3
## bu neden 1 varken 2'yi de aldi amk

{'ENSG00000069667.16': [1, 1, 0.9961195, 4],
 'ENSG00000169071.15': [1, 3, 0.9767402500000001, 4],
 'ENSG00000185483.13': [1, 3, 0.9950012000000001, 5]}

In [70]:
list(a1)

['ENSG00000069667.16', 'ENSG00000169071.15', 'ENSG00000185483.13']

In [71]:
list(a2)

['ENSG00000169071.15', 'ENSG00000185483.13', 'ENSG00000069667.16']

In [60]:
[(i,j) for i in a1 for j in a2 for k in a3 if i!=j or j!=k or i!=k]

[('ENSG00000069667.16', 'ENSG00000169071.15'),
 ('ENSG00000069667.16', 'ENSG00000169071.15'),
 ('ENSG00000069667.16', 'ENSG00000169071.15'),
 ('ENSG00000069667.16', 'ENSG00000185483.13'),
 ('ENSG00000069667.16', 'ENSG00000185483.13'),
 ('ENSG00000069667.16', 'ENSG00000185483.13'),
 ('ENSG00000069667.16', 'ENSG00000069667.16'),
 ('ENSG00000069667.16', 'ENSG00000069667.16'),
 ('ENSG00000169071.15', 'ENSG00000169071.15'),
 ('ENSG00000169071.15', 'ENSG00000169071.15'),
 ('ENSG00000169071.15', 'ENSG00000185483.13'),
 ('ENSG00000169071.15', 'ENSG00000185483.13'),
 ('ENSG00000169071.15', 'ENSG00000185483.13'),
 ('ENSG00000169071.15', 'ENSG00000069667.16'),
 ('ENSG00000169071.15', 'ENSG00000069667.16'),
 ('ENSG00000169071.15', 'ENSG00000069667.16'),
 ('ENSG00000185483.13', 'ENSG00000169071.15'),
 ('ENSG00000185483.13', 'ENSG00000169071.15'),
 ('ENSG00000185483.13', 'ENSG00000169071.15'),
 ('ENSG00000185483.13', 'ENSG00000185483.13'),
 ('ENSG00000185483.13', 'ENSG00000185483.13'),
 ('ENSG000001

In [63]:
b1= tt.scorer_for_1_to_n(a1, return_path=False, return_best=False)
b2= tt.scorer_for_1_to_n(a2, return_path=False, return_best=False)
b3= tt.scorer_for_1_to_n(a3, return_path=False, return_best=False)
_=[(b1[i], b2[j], b3[k]) for i in b1 for j in b2 for k in b3 if i!=j or j!=k or i!=k]

In [65]:
import operator
c = [list(map(operator.add, map(operator.add, b1[i], b2[j]), b3[k])) + [i, j, k] for i in b1 for j in b2 for k in b3 if i!=j or j!=k or i!=k]
np.array(c)

array([['-3', '-3', '2.9689792500000003', '19', '0', '36',
        'ENSG00000069667.16', 'ENSG00000169071.15', 'ENSG00000069667.16'],
       ['-3', '-5', '2.9496', '17', '0', '30', 'ENSG00000069667.16',
        'ENSG00000169071.15', 'ENSG00000169071.15'],
       ['-3', '-5', '2.9678609500000004', '15', '0', '25',
        'ENSG00000069667.16', 'ENSG00000169071.15', 'ENSG00000185483.13'],
       ['-3', '-5', '2.9872402000000005', '17', '0', '31',
        'ENSG00000069667.16', 'ENSG00000185483.13', 'ENSG00000069667.16'],
       ['-3', '-7', '2.9678609500000004', '15', '0', '25',
        'ENSG00000069667.16', 'ENSG00000185483.13', 'ENSG00000169071.15'],
       ['-3', '-7', '2.9861219000000006', '13', '0', '20',
        'ENSG00000069667.16', 'ENSG00000185483.13', 'ENSG00000185483.13'],
       ['-3', '-5', '2.9689792500000003', '19', '0', '36',
        'ENSG00000069667.16', 'ENSG00000069667.16', 'ENSG00000169071.15'],
       ['-3', '-5', '2.9872402000000005', '17', '0', '31',
        'ENSG00

In [69]:
sorted(c, reverse=True)

[[-3,
  -3,
  2.9872402000000005,
  17,
  0,
  31,
  'ENSG00000185483.13',
  'ENSG00000069667.16',
  'ENSG00000069667.16'],
 [-3,
  -3,
  2.9689792500000003,
  19,
  0,
  36,
  'ENSG00000069667.16',
  'ENSG00000169071.15',
  'ENSG00000069667.16'],
 [-3,
  -3,
  2.9678609500000004,
  15,
  0,
  25,
  'ENSG00000185483.13',
  'ENSG00000169071.15',
  'ENSG00000069667.16'],
 [-3,
  -5,
  2.9872402000000005,
  17,
  0,
  31,
  'ENSG00000069667.16',
  'ENSG00000185483.13',
  'ENSG00000069667.16'],
 [-3,
  -5,
  2.9872402000000005,
  17,
  0,
  31,
  'ENSG00000069667.16',
  'ENSG00000069667.16',
  'ENSG00000185483.13'],
 [-3,
  -5,
  2.9861219000000006,
  13,
  0,
  20,
  'ENSG00000185483.13',
  'ENSG00000185483.13',
  'ENSG00000069667.16'],
 [-3,
  -5,
  2.9861219000000006,
  13,
  0,
  20,
  'ENSG00000185483.13',
  'ENSG00000069667.16',
  'ENSG00000185483.13'],
 [-3,
  -5,
  2.9689792500000003,
  19,
  0,
  36,
  'ENSG00000169071.15',
  'ENSG00000069667.16',
  'ENSG00000069667.16'],
 [-3,
  

In [15]:
rc37 = Dataset(dm.change_server(37), narrow_search=False).initialize_external_conversion()
#rc38 = Dataset(dm.change_server(38), narrow_search=False).initialize_external_conversion()

2022-08-12 22:26:23 INFO:dataset: Comparison data frame is being constructed for releases: [107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79].


In [16]:
d=np.unique(rc37['id_db'])

In [17]:
[i for i in d if "YX65C7" in i]

['XXyac-YX65C7_A.2', 'XXyac-YX65C7_A.3', 'XXyac-YX65C7_A.4']

In [18]:
"PRED58" in d

True

In [19]:
amk = [i for i in a if i not in d]

In [20]:
print(len([i for i in a if i in d]))
print(len(amk))

58
113


In [21]:
np.array([i for i in amk if i.split('.')[0] not in d])

array(['RP11-442N24--B.1', 'RP11-99J16--A.2', 'RP11-59D5--B.2',
       'RP11-445L13--B.3', 'RP11-544L8--B.4', 'XXyac-YX65C7-A.2',
       'XXyac-YX65C7-A.3', 'RP11-524D16--A.3', 'RP11-453F18--B.1',
       'Metazoa-SRP', 'Y-RNA', 'XX-DJ76P10--A.2', '5S-rRNA', 'Y-RNA.1',
       'Y-RNA.2', 'RP11-1157N2--B.2', 'RP1-213J1P--B.1',
       'RP1-213J1P--B.2', 'RP4-633O19--A.1', 'RP4-754E20--A.5',
       'CTA-280A3--B.2'], dtype='<U16')

In [22]:
# sonra da combinatorial olarak _ to - degisikligi yap..
 
# np.array([i for i in amk if i.split('.')[0] not in d])

In [23]:
jkl = dict()
askdnmkajsdn = rc37["name_db"]
for ind, i in enumerate([i for i in a if i in d]):
    print(ind, end=",")
    jkl[i] = set(askdnmkajsdn[rc37["id_db"]==i])


0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,

In [24]:
np.unique(rc37[rc37["id_db"] == 'FLJ27365']["name_db"])

array(['UniProtKB Gene Name', 'Uniprot_gn'], dtype=object)

In [26]:
ex=dm.create_external_all()

In [36]:
np.unique(ex[ex['assembly'] == 37]['name_db'])

array(['Clone_based_ensembl_gene', 'Clone_based_vega_gene', 'Havana gene',
       'RFAM'], dtype=object)

array(['RFAM'], dtype=object)

In [57]:
jkl = dict()
askdnmkajsdn = rc37["name_db"]
for ind, i in enumerate(a):
    print(ind, end=",")
    jkl[i] = set(askdnmkajsdn[rc37["id_db"]==i])


0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,27

In [59]:
Counter([j for i in jkl for j in jkl[i]])

Counter({'Clone-based (Vega) gene': 377,
         'Clone_based_vega_gene': 377,
         'Clone-based (Vega)': 377,
         'Clone-based (Ensembl) gene': 87,
         'Clone-based (Ensembl)': 87,
         'Clone_based_ensembl_gene': 87,
         'EntrezGene': 41,
         'Uniprot_gn': 42,
         'HGNC Symbol': 20,
         'NCBI gene (formerly Entrezgene)': 41,
         'WikiGene': 17,
         'UniProtKB Gene Name': 42,
         'RFAM': 2})

In [6]:
df=dm.create_external_all()

In [5]:
ExternalDatabases(dm).give_list_for_case(give_type="db")

['EntrezGene',
 'UniProtKB Gene Name',
 'NCBI gene (formerly Entrezgene)',
 'HGNC Symbol']

In [8]:
np.sum(df['assembly']==38)

202607

In [9]:
np.sum(df['assembly']==37)

76298

In [13]:
np.unique(df[df['assembly']==37]["name_db"])

array(['Clone_based_ensembl_gene', 'Clone_based_vega_gene', 'Havana gene'],
      dtype=object)

In [12]:
df[df['graph_id']=='ENSG00000173727.7']

,release,graph_id,id_db,name_db,ensembl_identity,xref_identity,assembly
202611,107,ENSG00000173727.7,AP000769.1,Clone_based_ensembl_gene,NaN,NaN,37


In [110]:
the_dict[db_name]["Assembly"]

{'37': {'Ensembl release': '79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107',
  'Include': False},
 '38': {'Ensembl release': '79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107',
  'Include': False}}

In [3]:

self = ExternalDatabases(dm)
give_type='assembly'
the_dict_loaded = self.load_modified_yaml()
the_dict = the_dict_loaded[self.db_manager.organism][self.db_manager.form]

result = set()
for db_name in the_dict:
    for asm in the_dict[db_name]["Assembly"]:
        item = the_dict[db_name]["Assembly"][asm]
        res_ens = map(int, item["Ensembl release"].split(","))
        if self.db_manager.ensembl_release in res_ens and item["Include"]:
            if give_type == "db" and int(asm) == self.db_manager.ensembl_mysql_server:
                result.add(db_name)
            elif give_type == "assembly":
                result.add(int(asm))
            else:
                raise ValueError
            
            
result

{37, 38}

In [7]:
item

{'Ensembl release': '79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107',
 'Include': False}

In [6]:

for i in dm.available_releases:
    ex = ExternalDatabases(dm.change_release(i))
    ass = ex.give_list_for_case(give_type="db")
    print(f"Rel:{i}, Ass{ass}")

Rel:79, Ass['EntrezGene', 'Vega gene', 'Clone_based_ensembl_gene', 'UniProtKB Gene Name', 'Vega_gene', 'HGNC Symbol', 'Havana gene', 'Clone_based_vega_gene']
Rel:80, Ass['EntrezGene', 'Vega gene', 'Clone_based_ensembl_gene', 'UniProtKB Gene Name', 'Vega_gene', 'HGNC Symbol', 'Havana gene', 'Clone_based_vega_gene']
Rel:81, Ass['EntrezGene', 'Vega gene', 'Clone_based_ensembl_gene', 'UniProtKB Gene Name', 'Vega_gene', 'HGNC Symbol', 'Havana gene', 'Clone_based_vega_gene']
Rel:82, Ass['EntrezGene', 'Vega gene', 'Clone_based_ensembl_gene', 'UniProtKB Gene Name', 'Vega_gene', 'HGNC Symbol', 'Havana gene', 'Clone_based_vega_gene']
Rel:83, Ass['EntrezGene', 'Vega gene', 'Clone_based_ensembl_gene', 'UniProtKB Gene Name', 'Vega_gene', 'HGNC Symbol', 'Havana gene', 'Clone_based_vega_gene']
Rel:84, Ass['EntrezGene', 'Vega gene', 'Clone_based_ensembl_gene', 'UniProtKB Gene Name', 'Vega_gene', 'HGNC Symbol', 'Havana gene', 'Clone_based_vega_gene']
Rel:85, Ass['EntrezGene', 'Vega gene', 'Clone_based_

In [68]:
kalds=[i for i in jkl if len(jkl[i]) == 0]
len(kalds)

113

In [37]:
bok = rc37[rc37["id_db"].isin(a)]

In [35]:
dsd = []
for i in a:
    c =set([j for j in np.unique(bok[bok["id_db"] == i]["name_db"])])
    #print(c)
    dsd.extend(c)
from collections import Counter
Counter(dsd).most_common()

[('Clone-based (Vega) gene', 377),
 ('Clone_based_vega_gene', 377),
 ('Clone-based (Vega)', 377),
 ('Clone-based (Ensembl) gene', 87),
 ('Clone-based (Ensembl)', 87),
 ('Clone_based_ensembl_gene', 87),
 ('Uniprot_gn', 42),
 ('UniProtKB Gene Name', 42),
 ('EntrezGene', 41),
 ('NCBI gene (formerly Entrezgene)', 41),
 ('HGNC Symbol', 20),
 ('WikiGene', 17),
 ('RFAM', 2)]